# Simple Prophet Examples

In [2]:
%load_ext autoreload

%autoreload 2

In [3]:

import numpy as np
import pandas as pd
import os 
import fbprophet

# help ipython find our path
directory = os.path.dirname(os.path.dirname(os.path.abspath('')))
os.chdir(directory)

import forecastframe as ff


## Non-Hierarchical Example

In [4]:
prophet_data = pd.read_csv("https://raw.githubusercontent.com/facebook/prophet/master/examples/example_retail_sales.csv")

In [5]:
prophet_data

,ds,y
0,1992-01-01,146376
1,1992-02-01,147079
2,1992-03-01,159336
3,1992-04-01,163669
4,1992-05-01,170068
...,...,...
288,2016-01-01,400928
289,2016-02-01,413554
290,2016-03-01,460093
291,2016-04-01,450935


In [6]:
fframe = ff.ForecastFrame(data=prophet_data, target="y", datetime_column="ds")

In [9]:
fframe

                 y
ds                
1992-01-01  146376
1992-02-01  147079
1992-03-01  159336
1992-04-01  163669
1992-05-01  170068
...            ...
2016-01-01  400928
2016-02-01  413554
2016-03-01  460093
2016-04-01  450935
2016-05-01  471421

[293 rows x 1 columns]

In [15]:
fframe.predict(model="prophet", future_periods=10, weekly_seasonality=True, daily_seasonality=True)
fframe.get_predictions()

,y,predicted_y,trend,predicted_y_upper,predicted_y_lower
ds,,,,,
1992-01-01,146376.0,131335.258888,149250.780024,140599.402603,121799.380356
1992-02-01,147079.0,131131.825070,150289.708735,140491.562645,122222.301265
1992-03-01,159336.0,164783.266318,151261.609788,173739.039895,155522.766208
1992-04-01,163669.0,164876.320165,152300.538499,174405.435694,155210.793484
1992-05-01,170068.0,179316.575111,153305.953381,188614.334402,170259.771207
...,...,...,...,...,...
2016-05-07,NaN,474009.909550,448979.625130,483199.929555,464815.717712
2016-05-08,NaN,472531.169543,449025.071656,482264.272084,462913.298661
2016-05-09,NaN,477923.719431,449070.518182,487200.490366,468654.923270


In [17]:
fframe.get_errors()
# fframe.get_errors()

,Actuals,Predictions,Absolute Percent Error,Absolute Error,Squared Error
count,293.000000,293.000000,293.000000,293.000000,2.930000e+02
mean,308971.310580,308971.430803,0.019505,5671.404632,5.252161e+07
std,86084.323897,85622.285250,0.016581,4519.567783,8.274258e+07
min,146376.000000,131131.825070,0.000001,0.331767,1.100694e-01
25%,234503.000000,235586.265327,0.008071,2234.232871,4.991797e+06
50%,314051.000000,318376.545866,0.015901,4640.050641,2.153007e+07
75%,375795.000000,370050.289741,0.024941,8033.921990,6.454390e+07
max,518253.000000,507987.462463,0.108426,22100.229318,4.884201e+08


In [35]:
fframe.cross_validate(folds=3, model="prophet")

df     index         ds       y  floor         t  y_scaled
0       0 1992-01-01  146376      0  0.000000  0.540025
1       1 1992-02-01  147079      0  0.013945  0.542619
2       2 1992-03-01  159336      0  0.026991  0.587839
3       3 1992-04-01  163669      0  0.040936  0.603824
4       4 1992-05-01  170068      0  0.054431  0.627432
..    ...        ...     ...    ...       ...       ...
69     69 1997-10-01  232251      0  0.944669  0.856844
70     70 1997-11-01  228284      0  0.958614  0.842209
71     71 1997-12-01  271054      0  0.972110  1.000000
72     72 1998-01-01  207853      0  0.986055  0.766832
73     73 1998-02-01  203863      0  1.000000  0.752112

[74 rows x 6 columns]
df     index         ds       y  floor         t  y_scaled
0       0 1992-01-01  146376      0  0.000000  0.540025
1       1 1992-02-01  147079      0  0.013945  0.542619
2       2 1992-03-01  159336      0  0.026991  0.587839
3       3 1992-04-01  163669      0  0.040936  0.603824
4       4 1992-05-0

In [39]:
# check that the latest round of predictions stored in the object were created using the params found in cross-validation
assert fframe.model_object.seasonality_mode == fframe.cross_validations[-1]["best_params"]["seasonality_mode"]

In [40]:
fframe.get_cross_validation_errors()

[{'In-Sample':              Actuals    Predictions  Absolute Percent Error  Absolute Error  \
  count      74.000000      74.000000               74.000000       74.000000   
  mean   198509.472973  198491.867873                0.050104    10083.191491   
  std     26907.419353   22375.499866                0.049940    10943.462357   
  min    146376.000000  155959.330002                0.000153       26.167567   
  25%    176693.500000  181545.230686                0.012393     2298.031114   
  50%    199172.500000  197959.748690                0.028598     5685.623925   
  75%    217961.000000  214673.898390                0.074053    14046.914810   
  max    271054.000000  240194.669356                0.179570    43296.390865   
  
         Squared Error  
  count   7.400000e+01  
  mean    2.198117e+08  
  std     4.325281e+08  
  min     6.847415e+02  
  25%     5.281192e+06  
  50%     3.232724e+07  
  75%     1.973184e+08  
  max     1.874577e+09  ,
  'Out-of-Sample':           

In [41]:
fframe.cross_validations[0]["test"]

,ds,y,predicted_y
0,1998-03-01,230313,225889.467947
1,1998-04-01,234503,234661.483315
2,1998-05-01,245027,235414.358755
3,1998-06-01,244067,239774.172447
4,1998-07-01,241431,248628.421302
...,...,...,...
68,2003-11-01,301265,305004.513365
69,2003-12-01,357577,302976.983224
70,2004-01-01,281460,303511.437231
71,2004-02-01,282444,294805.307417
